In [1]:
import UWDiffusion as DIF
import underworld3 as uw
import sympy as sp

In [2]:
D12 = uw.options.getReal(name='D_12', default = 0.833)
D13 = uw.options.getReal(name='D_13', default = 0.833)
D23 = uw.options.getReal(name='D_23', default = 0.168)

In [3]:
mesh = uw.meshing.StructuredQuadBox( minCoords=(0, 0), maxCoords=(1, 1), elementRes=(50,50) )

Structured box element resolution 50 50


In [4]:
MC_diffusion = DIF.MulticomponentDiffusionModel(n_components=3, mesh=mesh, order=2)

In [6]:
D_matrix = DIF.diffusion_matrices.Maxwell_Stefan_diffusion_matrix(n=3)

In [7]:
D_matrix

Matrix([
[(D_12*D_13*D_21*\xi_1 - D_12*D_13*D_21 - D_12*D_13*D_23*\xi_1)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2),                  (D_12*D_21*D_23*\xi_1 - D_13*D_21*D_23*\xi_1)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2)],
[                 (D_12*D_13*D_21*\xi_2 - D_12*D_13*D_23*\xi_2)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2), (D_12*D_21*D_23*\xi_2 - D_12*D_21*D_23 - D_13*D_21*D_23*\xi_2)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2)]])

In [8]:
D_symbols = [sym for sym in D_matrix.free_symbols if sym.name.startswith(r"D")]
D_values = [D23, D12, D12, D13]

# Create a dictionary mapping each symbol to its corresponding value
substitution_dict = {symbol: value for symbol, value in zip(D_symbols, D_values)}

# Perform the substitution
D_matrix_substituted = D_matrix.subs(substitution_dict)

MC_diffusion.diffusivity_matrix = D_matrix

In [9]:
MC_diffusion.init_model()

IndexError: Index out of range: a[1]

In [ ]:
n_components = MC_diffusion.n_components
flux = []
for i in range(n_components-1):
    flux.append( MC_diffusion._kappa_history[i] * MC_diffusion._nabla_symbols[i] )

flux[0]

In [13]:
MC_diffusion._diffusivity_matrix * MC_diffusion._nabla_symbols[0:2, :]

Matrix([
[\nabla\xi_0*(D_12*D_13*D_21*\xi_1 - D_12*D_13*D_21 - D_12*D_13*D_23*\xi_1)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2) + \nabla\xi_1*(D_12*D_21*D_23*\xi_1 - D_13*D_21*D_23*\xi_1)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2)],
[\nabla\xi_0*(D_12*D_13*D_21*\xi_2 - D_12*D_13*D_23*\xi_2)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2) + \nabla\xi_1*(D_12*D_21*D_23*\xi_2 - D_12*D_21*D_23 - D_13*D_21*D_23*\xi_2)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2)]])

In [ ]:
MC_diffusion._nabla_symbols[0:2, :] * MC_diffusion._kappa_history[0]

In [ ]:
MC_diffusion._kappa_history[0] # *MC_diffusion._nabla_symbols[0:2, :]

In [14]:
MC_diffusion._diffusivity_matrix * MC_diffusion._nabla_symbols[0:MC_diffusion.n_components-1, :]

Matrix([
[\nabla\xi_0*(D_12*D_13*D_21*\xi_1 - D_12*D_13*D_21 - D_12*D_13*D_23*\xi_1)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2) + \nabla\xi_1*(D_12*D_21*D_23*\xi_1 - D_13*D_21*D_23*\xi_1)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2)],
[\nabla\xi_0*(D_12*D_13*D_21*\xi_2 - D_12*D_13*D_23*\xi_2)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2) + \nabla\xi_1*(D_12*D_21*D_23*\xi_2 - D_12*D_21*D_23 - D_13*D_21*D_23*\xi_2)/(D_12*D_21*\xi_1 + D_12*D_21*\xi_2 - D_12*D_21 - D_12*D_23*\xi_1 - D_13*D_21*\xi_2)]])

In [ ]:
for i in range(MC_diffusion.n_components):
    flux = MC_diffusion.mesh_vars[i].jacobian() * grad_matrix

In [ ]:
MC_diffusion.diffusivity_matrix

In [ ]:
MC_diffusion.mesh_vars[0].jacobian()